In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import math
import missingno as msno

# Funciones de Soporte (Helpers)

In [2]:
def map_languages(row) -> pd.Series:
    """
    En este método vamos a emular un OneHot encoding pero a mano.
    Recibe como parámetro el registro y devuelve el nuevo registro actualizado
    """
    # print(row['lenguajes_array'])
    languages = (row['lenguajes_array'])
    errors = 0
    try:
      for l in languages:
        if l != '':
          row[str.lower(l)] = 1
        else:
          errors += 1 # El lenguaje es '' string :(
    except:
      errors += 1

    # print('Count Errors:', errors)
    return row

# Trabajo Final

### Hipótesis

- Pregunta #1: Que salario o rango debería asignarle basado en mis variables?
    - Años de Experiencia / Seniority
    - Tecnologías (Lenguajes de Programación)
    - Lugar de Residencia
    - Dedicación
    - Estudios alcanzados
    - Posición
    - Tipo de Contrato

## Limpeza de Datos y Armado de DataSet (Data Wrangling)

In [3]:
# tomamos el DataSet y hacemos un poco de limpieza
#   -> Renombrar columnas para mejorar su legibilidad
#   -> Removemos algunas columnas que no son necesarios

df_salary = pd.read_csv('https://raw.githubusercontent.com/mauriciobergallo/coderhouse-datascience/refs/heads/main/datasets/encuesta-salarial-2024.csv', skiprows=0,
                        thousands=".",
                        decimal=",",)
df_salary.rename(columns={'Unnamed: 0': 'indice', '_sal': 'salario', \
    'lenguajes_de_programacion_o_tecnologias_que_utilices_en_tu_puesto_actual': 'lenguajes', \
    'donde_estas_trabajando': 'lugar_residencia', 'trabajo_de': 'posicion'}, inplace=True)
df_salary.drop(columns={'indice'}, inplace=True)

df_salary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5490 entries, 0 to 5489
Data columns (total 48 columns):
 #   Column                                                                                Non-Null Count  Dtype  
---  ------                                                                                --------------  -----  
 0   lugar_residencia                                                                      5490 non-null   object 
 1   dedicacion                                                                            5490 non-null   object 
 2   tipo_de_contrato                                                                      5490 non-null   object 
 3   ultimo_salario_mensual_o_retiro_bruto_en_pesos_argentinos                             5490 non-null   int64  
 4   ultimo_salario_mensual_o_retiro_neto_en_pesos_argentinos                              5214 non-null   float64
 5   pagos_en_dolares                                                                   

In [4]:
# Removemos algunas columnas que no vamos a tener en cuenta
df_salary.drop(columns={
    'bases_de_datos',
    'ultimo_salario_mensual_o_retiro_bruto_en_pesos_argentinos',
    'ultimo_salario_mensual_o_retiro_neto_en_pesos_argentinos',
    'pagos_en_dolares',
    'si_tu_sueldo_esta_dolarizado_cual_fue_el_ultimo_valor_del_dolar_que_tomaron',
    'recibis_algun_tipo_de_bono',
    'a_que_esta_atado_el_bono',
    'tuviste_actualizaciones_de_tus_ingresos_laborales_durante_2024',
    'de_que_fue_el_ajuste_total_acumulado',
    'en_que_mes_fue_el_ultimo_ajuste',
    'como_consideras_que_estan_tus_ingresos_laborales_comparados_con_el_semestre_anterior',
    'contas_con_beneficios_adicionales',
    'que_tan_conforme_estas_con_tus_ingresos_laborales',
    'estas_buscando_trabajo',
    'antiguedad_en_la_empresa_actual',
    'cuantas_personas_tenes_a_cargo',
    'plataformas_que_utilizas_en_tu_puesto_actual',
    'frameworksherramientas_y_librerias_que_utilices_en_tu_puesto_actual',
    'qa_testing',
    'cantidad_de_personas_en_tu_organizacion',
    'modalidad_de_trabajo',
    'si_trabajas_bajo_un_esquema_hibrido_cuantos_dias_a_la_semana_vas_a_la_oficina',
    'en_los_ultimos_6_mesesse_aplico_alguna_politica_de_ajustes_salariales',
    'en_los_ultimos_6_meseshubo_reduccion_de_personal',
    'la_recomendas_como_un_buen_lugar_para_trabajar',
    'que_tanto_estas_usando_copilotchatgpt_u_otras_herramientas_de_ia_para_tu_trabajo',
    'anos_en_el_puesto_actual',
    'salir_o_seguir_contestando',
    'estado',
    'carrera',
    'institucion_educativa',
    'salir_o_seguir_contestando_sobre_las_guardias',
    'tenes_guardias',
    'cuanto_cobras_por_guardia',
    'aclara_el_numero_que_ingresaste_en_el_campo_anterior',
    'salir_o_seguir_contestando_sobre_estudios',
    'tengo_edad',
    'genero',
    'sueldo_dolarizado'
}, inplace=True)

df_salary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5490 entries, 0 to 5489
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   lugar_residencia          5490 non-null   object
 1   dedicacion                5490 non-null   object
 2   tipo_de_contrato          5490 non-null   object
 3   posicion                  5490 non-null   object
 4   anos_de_experiencia       5490 non-null   int64 
 5   lenguajes                 5488 non-null   object
 6   maximo_nivel_de_estudios  2567 non-null   object
 7   seniority                 5490 non-null   object
 8   salario                   5490 non-null   int64 
dtypes: int64(2), object(7)
memory usage: 386.1+ KB


In [5]:
# Categorizar columnas
#   -> Dedicación
#   -> TipoDeContrato
#   -> MaximoNivelDeEstudio
#   -> Seniority
df_salary['dedicacion'] = df_salary['dedicacion'].astype('category')
df_salary['tipo_de_contrato'] = df_salary['tipo_de_contrato'].astype('category')
df_salary['maximo_nivel_de_estudios'] = df_salary['maximo_nivel_de_estudios'].astype('category')
df_salary['seniority'] = df_salary['seniority'].astype('category')
df_salary['lugar_residencia'] = df_salary['lugar_residencia'].astype('category')

df_salary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5490 entries, 0 to 5489
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype   
---  ------                    --------------  -----   
 0   lugar_residencia          5490 non-null   category
 1   dedicacion                5490 non-null   category
 2   tipo_de_contrato          5490 non-null   category
 3   posicion                  5490 non-null   object  
 4   anos_de_experiencia       5490 non-null   int64   
 5   lenguajes                 5488 non-null   object  
 6   maximo_nivel_de_estudios  2567 non-null   category
 7   seniority                 5490 non-null   category
 8   salario                   5490 non-null   int64   
dtypes: category(5), int64(2), object(2)
memory usage: 200.0+ KB


In [6]:
# Verificamos la cantidad de NaN en el DataFrame 
df_salary.isna().sum()

lugar_residencia               0
dedicacion                     0
tipo_de_contrato               0
posicion                       0
anos_de_experiencia            0
lenguajes                      2
maximo_nivel_de_estudios    2923
seniority                      0
salario                        0
dtype: int64

Nota: Vemos que hay 2 NaN para 'lenguajes' y 2923 para 'maximo_nivel_de_estudio',  por lo cual usaremos dos técnicas distintas, eliminar los registros para los cuales lenguajes es NaN (sabemos que no nos sirven debido a que vamos a utilizar los lenguajes como campo para predecir), y en el caso de 'maximmo_nivel_de_estudios' tomamos la MODA de la columna. 

In [7]:
# Mostramos las filas a borrar
df_filas_a_borrar = df_salary.loc[df_salary['lenguajes'].isna()]
df_filas_a_borrar

,lugar_residencia,dedicacion,tipo_de_contrato,posicion,anos_de_experiencia,lenguajes,maximo_nivel_de_estudios,seniority,salario
2195,Ciudad Autónoma de Buenos Aires,Full-Time,Staff (planta permanente),Infosec,1,NaN,Universitario,Junior,1371000
4163,Ciudad Autónoma de Buenos Aires,Full-Time,Staff (planta permanente),Scrum Master,10,NaN,NaN,Senior,3109560


In [8]:
# Eliminamos dichas filas
df_filas_a_borrar.index
df_salary.drop(index=df_filas_a_borrar.index, inplace=True)

In [9]:
# Calculamos la moda de 'maximo_nivel_de_educacion' para reemplazar en los valores nulos 
df_salary['maximo_nivel_de_estudios'].mode()

0    Universitario
Name: maximo_nivel_de_estudios, dtype: category
Categories (8, object): ['Doctorado', 'Maestría', 'Posdoctorado', 'Posgrado/Especialización', 'Primario', 'Secundario', 'Terciario', 'Universitario']

In [14]:
# Reemplazamos los valores NaN con la MODA
df_salary.fillna(value={ 'maximo_nivel_de_estudios': 'Universitario' }, inplace=True)

In [15]:
# Volvemos a listar los NaN
df_salary.isna().sum()

lugar_residencia            0
dedicacion                  0
tipo_de_contrato            0
posicion                    0
anos_de_experiencia         0
lenguajes                   0
maximo_nivel_de_estudios    0
seniority                   0
salario                     0
dtype: int64

In [16]:
# Filtramos posiciones relacionadas a desarrollo web

profession_filter = ['Developer', 'Technical Leader', 'Data Engineer', 'Automatizador', \
    'Consultant', 'Data Scientist', 'Architect', 'Engineer', 'Machine Learning Engineer', \
        'Analista Senior en Informatica', 'AI Engineer', 'Operaciones', \
            'Hago desde el análisis y la programación. Diseño de DB, etc', 'Analista Tecnico Funcional', \
                'Ingenieria Electronica', 'Técnico en informática']

df_salary.where(df_salary['posicion'].isin(profession_filter), inplace=True)

# Reindexamos para evitar problemas de índice
df_salary = df_salary.dropna().reset_index(drop=True)
df_salary.tail()


,lugar_residencia,dedicacion,tipo_de_contrato,posicion,anos_de_experiencia,lenguajes,maximo_nivel_de_estudios,seniority,salario
2953,Ciudad Autónoma de Buenos Aires,Full-Time,Staff (planta permanente),Data Engineer,5.0,"Bash/Shell, Python, Scala",Universitario,Semi-Senior,2200000.0
2954,Ciudad Autónoma de Buenos Aires,Part-Time,Staff (planta permanente),Developer,4.0,"Bash/Shell, CSS, HTML, Java, Javascript, Scala...",Universitario,Semi-Senior,1700000.0
2955,Buenos Aires,Full-Time,Tercerizado (trabajo a través de consultora o ...,Developer,5.0,"CSS, HTML, Javascript, TypeScript",Terciario,Semi-Senior,2500000.0
2956,Ciudad Autónoma de Buenos Aires,Full-Time,Staff (planta permanente),Technical Leader,30.0,SQL,Universitario,Senior,4000000.0
2957,Buenos Aires,Full-Time,Staff (planta permanente),Developer,5.0,"HTML, Javascript, PHP, Rust, SQL",Universitario,Semi-Senior,1290000.0


In [17]:
# Actualmente los lenguajes de programación vienen en un string separados por coma y espacio (', '); por ende, vamos a separarlos en un array y 
# usarlos como columnas

# Para ello:
# Spliteamos los lenguajes de programación en un array dentro de la columna nueva lenguajes_array
df_salary['lenguajes_array'] = df_salary['lenguajes'] \
    .apply(func=(lambda x: str(x).split(', ')))

# Con este pedazo de código lo que hacemos es crear las columnas de los lenguajes de programación,
# además lo limpiamos un poco
languages_list = df_salary['lenguajes_array'].to_list()

df_languages = pd.DataFrame(languages_list)
unique_languages = pd.unique(np.char.lower(df_languages.stack().to_numpy().tolist()))

df_unique_languages = pd.DataFrame(unique_languages).replace('', np.nan).dropna()
unique_languages_list = pd.unique(np.char.lower(df_unique_languages.stack().to_numpy().tolist()))

lenguages_columns_length = unique_languages_list.shape[0]
lenguages_rows_length = df_salary.shape[0]

languages_df = pd.DataFrame(np.zeros((lenguages_rows_length, lenguages_columns_length)), columns=unique_languages_list)
final_df = pd.concat([df_salary, languages_df], axis=1)

In [18]:
# Aplicamos la función map_languages, que simula un OneHotEncoder a mano
final_df_encoded = final_df.apply(map_languages, axis=1)

# Mostramos un poco los resultados...
final_df_encoded[['lenguajes', 'html', 'javascript', 'c#', 'seniority', 'anos_de_experiencia', 'tipo_de_contrato', 'salario']].head()

# Creo un nuevo CSV con las columnas para verlas mejor y poder elegir cual eliminar y cual no
# df_columns_to_clean = final_df_encoded.columns.to_frame()
# df_columns_to_clean.to_csv('./datasets/coolumns_for_cleaning.csv')

# Elimino nuevamente algunas columnas que considero "no útiles"
final_df_encoded.drop(columns={
    'ninguno de los anteriores',
    'relacionados a salesforce: apex',
    'una poronga',
    'sap pi po is',
    'tal',
    'edi',
    'microsoft excel',
    'openedge abl',
    'soql ',
    'no trabajo con lenguajes sino que realizo diseño de arquitecturas a alto nivel',
    'no hago programacion',
    'wikimedia',
    'actualmente me encuentro entre asignaciones',
    'pero mi perfil en la empresa es de java y angularts',
    'vb .net  y vb .6',
    'lenguajes', 
    'lenguajes_array'
}, inplace=True)

# Vuelvo a Eliminar NaN en caso de que me hayan quedado
final_df_encoded = final_df_encoded.dropna().reset_index(drop=True)


## Informe sobre el DataSet ya limpio

In [19]:
# Asignación
df = final_df_encoded

In [20]:
# Forma
print(df.shape)

(2958, 125)


In [21]:
# Describimos el DataSet
df.describe().T

,count,mean,std,min,25%,50%,75%,max
anos_de_experiencia,2958.0,7.323191e+00,6.625981e+00,0.0,3.0,5.0,10.0,3.900000e+01
salario,2958.0,6.130852e+06,3.502636e+07,120000.0,1150000.0,1850000.0,3000000.0,1.105953e+09
html,2958.0,3.820149e-01,4.859623e-01,0.0,0.0,0.0,1.0,1.000000e+00
javascript,2958.0,5.463151e-01,4.979345e-01,0.0,0.0,1.0,1.0,1.000000e+00
php,2958.0,1.260987e-01,3.320167e-01,0.0,0.0,0.0,0.0,1.000000e+00
...,...,...,...,...,...,...,...,...
sqr,2958.0,3.380663e-04,1.838658e-02,0.0,0.0,0.0,0.0,1.000000e+00
vbscript,2958.0,3.380663e-04,1.838658e-02,0.0,0.0,0.0,0.0,1.000000e+00
blue prism,2958.0,3.380663e-04,1.838658e-02,0.0,0.0,0.0,0.0,1.000000e+00
poweshell,2958.0,3.380663e-04,1.838658e-02,0.0,0.0,0.0,0.0,1.000000e+00


In [22]:
# Perimeros 10 Registros
df.head(10)

,lugar_residencia,dedicacion,tipo_de_contrato,posicion,anos_de_experiencia,maximo_nivel_de_estudios,seniority,salario,html,javascript,...,rexx,ssis,graphql,mulesoft,iron python,sqr,vbscript,blue prism,poweshell,classic asp
0,Ciudad Autónoma de Buenos Aires,Full-Time,Staff (planta permanente),Developer,3.0,Posgrado/Especialización,Semi-Senior,1700000.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Buenos Aires,Full-Time,Staff (planta permanente),Technical Leader,3.0,Universitario,Semi-Senior,2000000.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Córdoba,Full-Time,Contractor,Data Engineer,5.0,Universitario,Semi-Senior,1000000.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Ciudad Autónoma de Buenos Aires,Full-Time,Staff (planta permanente),Developer,9.0,Universitario,Senior,850000.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Ciudad Autónoma de Buenos Aires,Full-Time,Contractor,Technical Leader,5.0,Universitario,Semi-Senior,3300000.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Córdoba,Full-Time,Contractor,Automatizador,0.0,Universitario,Junior,450000.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,Córdoba,Full-Time,Staff (planta permanente),Technical Leader,9.0,Universitario,Senior,835469.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,Buenos Aires,Full-Time,Contractor,Technical Leader,3.0,Universitario,Semi-Senior,1050000.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,Ciudad Autónoma de Buenos Aires,Full-Time,Staff (planta permanente),Developer,4.0,Universitario,Semi-Senior,860000.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,Corrientes,Full-Time,Contractor,Developer,15.0,Posgrado/Especialización,Senior,4000000.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Últimos 10 Registros
df.tail(10)

In [ ]:
# Mostramos que no hay valores nulos o vacíos
msno.matrix(df)

In [ ]:
# Algunas columnas
print(df.columns)

## Análisis de variables Categóricas

- Por cada variable categórica mostramos la cantidad de dichas ocurrencias
- Además usamos Scatterplot para mostrar la distribución de los valores por cada categoría

In [ ]:
# Creamos un DF de soporte para poder mostrar los valores de Salario expresado en $1.000.000
# df_copy = df
# df_copy['salario_en_millones'] = df_copy['salario'] / 1000000
# df_copy[['salario', 'salario_en_millones']]
df['salario_en_millones'] = df['salario'] / 1000000
df[['salario', 'salario_en_millones']]

### Codificación de variables categóricas

Para un mejor análisis posterior (los algoritmos deben trabajar con valores numéricos), debemos codificar (encoding) de las variables categóricas. Utilizaremos el LabelEncoder debido a que no es la idea agregar mas columnas al DataSet.

In [ ]:
# Ya que son Strings (cadenas de caracteres), utilizamos LabelEncoder
encoder = LabelEncoder()
df['seniority_encoded'] = encoder.fit_transform(df['seniority'])
df['lugar_residencia_encoded'] = encoder.fit_transform(df['lugar_residencia'])
df['dedicacion_encoded'] = encoder.fit_transform(df['dedicacion'])
df['tipo_de_contrato_encoded'] = encoder.fit_transform(df['tipo_de_contrato'])
df['maximo_nivel_de_estudios_encoded'] = encoder.fit_transform(df['maximo_nivel_de_estudios'])
df[['seniority', 'seniority_encoded', \
    'lugar_residencia', 'lugar_residencia_encoded', \
    'dedicacion', 'dedicacion_encoded', \
    'tipo_de_contrato', 'tipo_de_contrato_encoded', \
    'maximo_nivel_de_estudios', 'maximo_nivel_de_estudios_encoded']]

### Variable: Lugar de Residencia

In [ ]:
df[['lugar_residencia', 'lugar_residencia_encoded']].value_counts()

In [ ]:
ax = sns.displot(data=df['lugar_residencia'])
ax.tick_params(axis='x', rotation=90)
plt.title('Cantidad de Developers por Residencia')
plt.show()

In [ ]:
#Scatterplot
ax = sns.scatterplot(df, x='lugar_residencia', y='salario_en_millones')
ax.tick_params(axis='x', rotation=90)
plt.title('Salarios en $1.000.000 por Residencia')
plt.show()

### Variable:Seniority

In [ ]:
print(df[['seniority', 'seniority_encoded']].value_counts())

In [ ]:
ax = sns.displot(data=df['seniority'])
plt.title('Cantidad de Developers por Seniority')
plt.show()

### Variable: Dedicación

In [ ]:
print(df[['dedicacion', 'dedicacion_encoded']].value_counts())

In [ ]:
ax = sns.displot(data=df['dedicacion'])
plt.title('Cantidad de Developers por Dedicación')
plt.show()

In [ ]:
ax = sns.scatterplot(df, x='dedicacion', y='salario_en_millones')
ax.tick_params(axis='x', rotation=90)
plt.title('Distribución de Salarios por Dedicación')
plt.show()

### Variable: Tipo de Contrato

In [ ]:
print(df[['tipo_de_contrato', 'tipo_de_contrato_encoded']].value_counts())

In [ ]:
ax = sns.displot(data=df['tipo_de_contrato'])
ax.tick_params(axis='x', rotation=90)
plt.title('Cantidad de Developers por Tipo de Contrato')
plt.show()

In [ ]:
ax = sns.scatterplot(df, x='tipo_de_contrato', y='salario_en_millones')
ax.tick_params(axis='x', rotation=90)
plt.title('Distribución de Salarios por Tipo de Contrato')
plt.show()

### Variable: Máximo nivel de Estudio alcanzado

In [ ]:
print(df[['maximo_nivel_de_estudios', 'maximo_nivel_de_estudios_encoded']].value_counts())

In [ ]:
ax = sns.displot(data=df['maximo_nivel_de_estudios'])
ax.tick_params(axis='x', rotation=90)
plt.title('Cantidad de Developers por Nivel de Estudio')
plt.show()

In [ ]:
ax = sns.scatterplot(df, x='maximo_nivel_de_estudios', y='salario_en_millones')
ax.tick_params(axis='x', rotation=90)
plt.title('Distribución de Salarios por Nivel De Estudio')
plt.show()

### Variable: Años de Experiencia

In [ ]:
print(df['anos_de_experiencia'].value_counts())

In [ ]:
plt.figure(dpi = 120, figsize= (10,20))
sns.displot(data=df['anos_de_experiencia'])
plt.title('Cantidad de Developers por Años de Experiencia')
plt.show()

In [ ]:
ax = sns.scatterplot(df, x='anos_de_experiencia', y='salario_en_millones')
ax.tick_params(axis='x', rotation=90)
plt.title('Distribución de Salarios por Años de Experiencia')
plt.show()

## Primer Corrección

Silvia Vilela:

Hola Mauricio felicitaciones, muy buen trabajo!!. La exploracion de los datos es exhaustiva y la presentacion del trabajo es muy profesional. Como comentarios a incluir que me parece pueden sumnar al proyuecto son: la presencia concreta de las preguntas (hay una hilo conductor en el trabajo pero no se observan las preguntas por ejemplo Se suele pagar mas en Buenos Aires, etc. Luego hay algunos casos que se rellenan valores por un valor en particulñar pero no se indica porque razon(usualmente debe ser por mean, por mayor ocurrencia del mismo, pero nunca en forma aleatoria sin justificacion). Luego al final deberia existir una conlusion final que englobe todas las respuetsas obtenidas (3 al menos deberian ser contestadas en el trabajo). Dejo todos los criterios completos y cumplidos, por favor inclui estas mejoras en el TP para que el profe pueda verlas completas en la entrega final. Muy buen Trabajo Mauricio y excelente tu participacion en clase!

Agregar:
- Preguntas a resolver.
- Conclusiones a cada paso.
- Explicar porque rellenamos los valores nulos
- Conclusión al final del EDA

## Manejo de Outliers

Se identifican los outliers en el DataFrame, se procede a eliminarlos debido a que son considerados "Bad Data"

In [ ]:
# Outliers

Q1 = df['salario_en_millones'].quantile(0.25)
Q3 = df['salario_en_millones'].quantile(0.75)
IQR = Q3 - Q1

threshold = 1.5
outliers = df[(df['salario_en_millones'] < Q1 - threshold * IQR) | (df['salario_en_millones'] > Q3 + threshold * IQR)]

df = df.drop(outliers.index)

## Mostramos nuevamente los gráficos por categoría

### Variable Seniority

In [ ]:
plt.subplots(figsize=(5, 5), dpi =200)
sns.set(style = "white")
ax = sns.boxplot(data=df, x=df['seniority'], y=df["salario_en_millones"])
plt.title('Salarios en $1.000.000 por Seniority')
plt.show()

### Variable Dedicación

In [ ]:
plt.subplots(figsize=(5, 5), dpi =200)
sns.set(style = "white")
ax = sns.boxplot(data=df, x=df['dedicacion'], y=df["salario_en_millones"])
plt.title('Salarios en $1.000.000 por Dedicación')
plt.show()

### Variable Tipo de Contrato

In [ ]:
plt.subplots(figsize=(5, 5), dpi =200)
sns.set(style = "white")
ax = sns.boxplot(data=df, x=df['tipo_de_contrato'], y=df["salario_en_millones"])
ax.tick_params(axis='x', rotation=90)
plt.title('Salarios en $1.000.000 por Dedicación')
plt.show()

### Variable Máximo nivel de estudio alcanzado

In [ ]:
plt.subplots(figsize=(5, 5), dpi =200)
sns.set(style = "white")
ax = sns.boxplot(data=df, x=df['maximo_nivel_de_estudios'], y=df["salario_en_millones"])
ax.tick_params(axis='x', rotation=90)
plt.title('Salarios en $1.000.000 por Nivel de Estudio alcanzado')
plt.show()

### Mapa de Calor para ver la relación entre dichas variables

Utilizamos un mapa de calor para intentar averiguar si hay alguna correlación entre las variables codificadas y nuestra variable target (Salario)


In [ ]:
#Correlaciones
plt.figure(dpi = 120,figsize= (5,4))
sns.heatmap(df[['seniority_encoded', 'lugar_residencia_encoded', 'dedicacion_encoded', \
    'tipo_de_contrato_encoded', 'maximo_nivel_de_estudios_encoded', 'salario']].corr(), fmt = ".2f", annot=True, lw=1, cmap = 'plasma')
plt.yticks(rotation = 0)
plt.xticks(rotation = 90)
plt.title('Correlación entre Variables Categóricas')
plt.show()

Conclusión: como se puede apreciar en el gráfico vemos que la feature SENIORITY es la que mas influye sobre el Salario, cosa que tiene sentido porque los salarios cambian mucho cuando un Developer es Senior / SemiSenior / Junior